### T-Testing ARAUS

**Remarkable Groups:**
- Pleasant = 5
- Eventful = 5
- Vibrant = 5

**Comparison Group** 
- Pleasant = 1


In [15]:
# Setup
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

file_path = "datasets/cleaned_ARAUS.csv"
data = pd.read_csv(file_path)

data.head()

,participant,fold_r,soundscape,masker,smr,stimulus_index,time_taken,is_attention,pleasant,eventful,...,M04000_0_r,M05000_0_r,M06300_0_r,M08000_0_r,M10000_0_r,M12500_0_r,M16000_0_r,M20000_0_r,Leq_L_r,Leq_R_r
0,ARAUS_00009,4,R0087_segment_binaural_44100_1.wav,silence_00004.wav,3,28,35.592,0,5,5,...,46.13,40.68,38.51,33.42,25.83,21.02,20.67,22.70,73.761966,75.353091
1,ARAUS_00021,1,R0081_segment_binaural_44100_2.wav,traffic_00029.wav,0,4,37.439,0,5,5,...,47.66,42.56,41.86,42.69,37.15,36.50,30.99,19.27,74.202644,73.493800
2,ARAUS_00021,1,R0046_segment_binaural_44100_2.wav,bird_00047.wav,3,8,37.833,0,5,5,...,43.73,39.67,35.29,33.46,27.51,19.27,18.67,12.37,67.246896,68.127026
3,ARAUS_00021,1,R0080_segment_binaural_44100_2.wav,traffic_00006.wav,3,11,33.782,0,5,5,...,44.57,43.30,43.81,36.82,31.24,28.05,23.03,17.66,67.395837,68.006605
4,ARAUS_00021,1,R0115_segment_binaural_44100_1.wav,bird_00059.wav,0,12,37.663,0,5,5,...,37.86,31.16,25.78,19.90,16.79,13.98,14.23,12.36,65.505416,66.575806


In [21]:
# Function to create groups with filters
def create_group(data : pd.DataFrame, filter_cols : list, filter_vals: list) -> pd.DataFrame:
    assert len(filter_cols) == len(filter_vals), "Filter columns and values must have the same length"

    group_data = data.copy()
    for col, val in zip(filter_cols, filter_vals):
        group_data = group_data[group_data[col] == val]
    return group_data


# Define groups with filters
# Format: {group_name: {"filter_cols": [col1, col2, ...], "filter_vals": [val1, val2, ...]}}
remarkable_groups = {
    "R_pleasant_5": {"filter_cols": ["pleasant"], "filter_vals": [5]},
    "R_eventful_5": {"filter_cols": ["eventful"], "filter_vals": [5]},
    "R_vibrant_5": {"filter_cols": ["vibrant"], "filter_vals": [5]},
}

comparison_groups = {
    "C_pleasant_1": {"filter_cols": ["pleasant"], "filter_vals": [1]},
    "C_eventful_1": {"filter_cols": ["eventful"], "filter_vals": [1]},
    "C_vibrant_1": {"filter_cols": ["vibrant"], "filter_vals": [1]},
}

#Define which columns from cleaned_ARAUS.csv to use for the analysis
# Format: {col_name in dataset: column name used for printing results}
columns_of_interest = {
    "Savg_r": "Average Sharpness (acum)",
    "Smax_r": "Peak Sharpness (acum)",
    "Navg_r": "Average Loudness (sone)",
    "Nmax_r": "Peak Loudness (sone)",
    "Favg_r": "Average Fluctuation Strength (vacil)",
    "Fmax_r": "Peak Fluctuation Strength (vacil)",
    "Ravg_r": "Average Roughness (asper)",
    "Rmax_r": "Peak Roughness (asper)",
    "Tavg_r": "Average Tonality (tonality units)",
    "Tmax_r": "Peak Tonality (tonality units)",
}
# Define which columns from cleaned_ARAUS.csv to include in ARAUS_relevant.csv
necessary_info = ["participant", "soundscape", "masker", "stimulus_index", "time_taken"]
relevant_data = data.loc[:, necessary_info + list(columns_of_interest.keys())]

# Write ARAUS csv with only relevant columns to new csv
relevant_data.to_csv("datasets/ARAUS_relevant.csv", index=False)

In [22]:
def process_groups(groups: dict, data: pd.DataFrame, columns: list) -> dict:
    """
    Creates groups from list of filters

    Returns:
      dict - "group names" : [relevant group data]
    """
    group_data = {}
    # items are the key value pairs of each group in dict
    for group_name, group_info in groups.items():
        # Create each group using the provided filters
        single_group_datum = create_group(
            data, group_info["filter_cols"], group_info["filter_vals"]
        )
        # Calculate statistics for each group and store them
        group_data[group_name] = single_group_datum[columns]
        # Write group data to csv
        single_group_datum[necessary_info + columns].to_csv(f"datasets/{group_name}.csv", index=False)
    return group_data


def calculate_statistics(group: pd.DataFrame):
    """Calculate mean, standard deviation and size all columns in a group"""
    return {
        "mean": group.mean(),
        "standard deviation": group.std(),
        "group size": len(group),
        "variance": group.var(),
        # Add more statistics as needed
    }

def process_statistics(group_data: dict):
    """Calculate statistics for all groups in a dictionary"""
    group_stats = {}
    for group_name, group in group_data.items():
        group_stats[group_name] = calculate_statistics(group)
    return group_stats


col_interest_titles = list(columns_of_interest.keys())

# # Process all groups and store their statistics
remarkable_group_data = process_groups(remarkable_groups, data, col_interest_titles)
comparison_group_data = process_groups(comparison_groups, data, col_interest_titles)
remarkable_group_stats = process_statistics(remarkable_group_data)
comparison_group_stats = process_statistics(comparison_group_data)

# print(remarkable_group_stats)
# print(comparison_group_stats)

In [23]:
def select_group(group_data: dict, group_name: str) -> dict:
    """
    Selects a group from the given group_data dictionary (which contains at least 1 group) and returns 
    a dictionary containing only the selected group's statistics or data.

    Works taking in both group_data and group_stats dictionaries.
    """
    return {group_name: group_data[group_name]}

def print_group_stats(group_stats: dict, columns_of_interest: dict, printable_stats: list = ["mean", "standard deviation", "group size"]):
    """
    Prints the statistics of each group in a formatted manner, using the labels from group_stats.
    """
    for group_name, stats in group_stats.items():
        group_size = stats["group size"]
        print(f"### Group: {group_name} - size: {group_size}")
        for col_key, label in columns_of_interest.items():
            for stat_key, stat_value in stats.items():
                if stat_key in printable_stats:
                    # Check if stat_value is a DataFrame or series
                    if isinstance(stat_value, (pd.DataFrame, pd.Series)):
                        formatted_label = f"{stat_key.capitalize()} {label}"
                        print(f"    - **{formatted_label}**: {stat_value[col_key]:.4f}")
                    else:  # For non-DataFrame/Series statistics (like 'size')
                        if stat_key != "group size":
                            formatted_label = f"{stat_key.capitalize()} of {group_name}"
                            print(f"    - **{formatted_label}**: {stat_value}")
            print()

        print("\n")


print(select_group(remarkable_group_stats, "R_pleasant_5"))
print("AND ITEMS BELOW")
print(remarkable_group_stats["R_pleasant_5"])
#print_group_stats(select_group(remarkable_group_stats, "R_pleasant_5"), columns_of_interest)

# print_group_stats(remarkable_group_stats, columns_of_interest)
#print_group_stats(comparison_group_stats, columns_of_interest)



{'R_pleasant_5': {'mean': Savg_r     1.401446
Smax_r     1.895222
Navg_r    12.932157
Nmax_r    20.431250
Favg_r     0.021601
Fmax_r     0.107063
Ravg_r     0.027561
Rmax_r     0.080859
Tavg_r     0.234625
Tmax_r     1.354519
dtype: float64, 'standard deviation': Savg_r    0.271485
Smax_r    0.416037
Navg_r    5.630724
Nmax_r    9.468881
Favg_r    0.020273
Fmax_r    0.052535
Ravg_r    0.007402
Rmax_r    0.050241
Tavg_r    0.263747
Tmax_r    0.908111
dtype: float64, 'group size': 2536, 'variance': Savg_r     0.073704
Smax_r     0.173087
Navg_r    31.705049
Nmax_r    89.659715
Favg_r     0.000411
Fmax_r     0.002760
Ravg_r     0.000055
Rmax_r     0.002524
Tavg_r     0.069563
Tmax_r     0.824665
dtype: float64}}
AND ITEMS BELOW
{'mean': Savg_r     1.401446
Smax_r     1.895222
Navg_r    12.932157
Nmax_r    20.431250
Favg_r     0.021601
Fmax_r     0.107063
Ravg_r     0.027561
Rmax_r     0.080859
Tavg_r     0.234625
Tmax_r     1.354519
dtype: float64, 'standard deviation': Savg_r    0.271485

In [24]:
from scipy.stats import ttest_ind


def significance_test(one_data: pd.DataFrame, two_data: pd.DataFrame) -> tuple:
    """
    Calculate the t-statistic and p-value for the two given groups.

    Args:
        one_data (pd.DataFrame): Dataframe containing data for the first group (with only relevant columns).
        two_data (pd.DataFrame): Dataframe containing data for the second group.


    Returns:
        tuple: A tuple of two dictionaries containing t-statistics and p-values for each column.
    """

    # Initializing dictionaries for results
    t_statistics = {}
    p_values = {}

    # Iterating over each statistic and calculating t-statistic and p-value
    for stat_name in one_data.columns:
        t_statistic, p_value = ttest_ind(
            one_data[stat_name], 
            two_data[stat_name], 
            equal_var=False, 
            nan_policy="omit"
        )
        t_statistics[stat_name] = t_statistic
        p_values[stat_name] = p_value

    return t_statistics, p_values



def verbose_compare_groups(group_one_data: dict, group_two_data: dict, columns_of_interest: dict, file, p_value : float = 0.01):
    """
    Compares two groups, prints the comparison of their means, and calculates the t-test for significant difference in means.

    Args:
        group_one_stats (dict): Dictionary containing statistics for the first group.
        group_two_stats (dict): Dictionary containing statistics for the second group.
        columns_of_interest (dict): Dictionary mapping data column keys to their descriptive labels.
        file (Python I/O file object): Open file that can be written to 
        p-value: Value for testing significance of t-tests (typically 0.05 or 0.01)
    """

    # Retrieve group names
    group_one_name, group_two_name = list(group_one_data.keys())[0], list(group_two_data.keys())[0]
    one_data, two_data = list(group_one_data.values())[0], list(group_two_data.values())[0]

    file.write(f"## COMPARISON BETWEEN {group_one_name} AND {group_two_name} GROUPS\n\n")

    means_one = calculate_statistics(one_data)['mean']
    means_two = calculate_statistics(two_data)['mean']

    # Perform t-test and compare means
    t_stats, p_values = significance_test(one_data, two_data)

    underscore_counter = 0

    # file.write comparison of means and summary of t-tests
    for col_key, label in columns_of_interest.items():
        group_one_stat = means_one[col_key]
        group_two_stat = means_two[col_key]

        higher_lower = "**HIGHER**" if group_one_stat > group_two_stat else "**LOWER**"
        inverse_higher_lower = "**LOWER**" if higher_lower == "**HIGHER**" else "**HIGHER**"

        file.write(f"### PARAMETER: {label}\n")
        file.write(f"- *{group_one_name}* MEAN: {group_one_stat:.4f} - {higher_lower}\n")
        file.write(f"- *{group_two_name}* MEAN: {group_two_stat:.4f} - {inverse_higher_lower}\n")

        significance = "**STATISTICALLY SIGNIFICANT**" if p_values[col_key] < p_value else "**NOT STATISTICALLY SIGNIFICANT**"
        file.write(f"- {significance} DIFFERENCE WITH P={p_value}: p-value: {p_values[col_key]:.4f}, t-value: {t_stats[col_key]:.4f}\n\n")

        underscore_counter += 1
        if underscore_counter % 2 == 0 and underscore_counter != 10:
            file.write("-------------------\n")



# Compare all remarkable and comparison groups, output to total_comparisons.md file
with open("t-test-outputs/total_comparisons.md", "w") as file:
    for r_group in remarkable_group_data:
        for c_group in comparison_group_data:
            verbose_compare_groups(
                select_group(remarkable_group_data, r_group),
                select_group(comparison_group_data, c_group),
                columns_of_interest,
                file
            )
            file.write("______________________________________\n\n\n")

